In [ ]:
# def Merge(dict1,dict2,dict3,dict4):
#     res={**dict1,**dict2,**dict3,**dict4}
#     return(res)


# def json_input(sheet,tags_dict,tags_dict2,tags_dict3):
    
#     values_list=sheet.row_values(7)
#     odWindowSize=values_list[5]
#     odAlpha=values_list[6]
#     odWindowPosition=values_list[7]
#     IntervalMins=values_list[8]
#     noise=values_list[9]
#     ssAlpha=values_list[11]
#     ssWindowPosition=values_list[12]
                
#     Configuration={
#         "MODEL_SERVICE_TOKEN": "<Token>",
#         "MODEL_SERVICE_ENDPOINT": "<URL>",
#         "DO_SSD_THRESHOLD": "0.9",
#         "INTERVAL": "30",
#         "SAMPLE_DATA_FREQUENCY": "2000",
#         "DELAY": "150",
#         "OD_window_size":odWindowSize,
#         "OD_alpha":odAlpha,
#         "OD_window_position":odWindowPosition,
#         "interpolated_minute": IntervalMins,
#         "noise":noise,
#         "SS_alpha":ssAlpha,
#         "SS_window_position":ssWindowPosition,      
        
#     }    
# #     print(Merge(Configuration,tags_dict,tags_dict2,tags_dict3))
#     main_config=Merge(Configuration,tags_dict,tags_dict2,tags_dict3)
    
#     return main_config
        

In [30]:
timeStamps=[]
values=[]
InputDataList=[]
main_input_dict_1={}
main_input_dict_2={}
ssd_dict={}
OutputDataList=[]

def inputDataList(critical_sensor_tag_values,edge_asset_values):
    pattern="%m-%d-%Y %H:%m:%s"

    data=pd.read_csv("suncorDATA.csv")
    
    
    
#     for total_tags in range(0,len(edge_asset_values)):
          
#         print(total_tags)
        
    for i,j in data.iterrows():            
        print(i)
        TagName=i[0]
#         print(edge_asset_values[total_tags].split(".")[0])
#             if TagName== edge_asset_values[total_tags] or TagName == edge_asset_values[total_tags].split(".")[0]+".STDEV":
        if TagName== "TagName":
            pass
        else:
            
            print("Fetching data for ",TagName)

            if i[1] == "TimeStamp":
                pass
            else:
                dates=pd.to_datetime(i[1])
    #             print((dates - pd.Timestamp("1970-01-01")) / pd.Timedelta('1s'))
                epoch=(dates - pd.Timestamp("1970-01-01")) / pd.Timedelta('1s')
                final_timestamp=str(epoch)
                timeStamps.append(str(epoch))
    #             print(i[2])
                final_value=float(i[2])
                values.append(i[2])

    #                 series_8=pd.Series(timeStamps)
    #             #     print(series_8)
    #                 series_9=pd.Series(values)

    #                 timeStampsToValueMap={"timestamp":series_8,"values":series_9}
    #                 dframe4= pd.DataFrame(timeStampsToValueMap)
    #                 tags_dict4=dframe4.set_index('timestamp')["values"].to_dict()
            main_tag=TagName.split(".")[0]
            if i[0].split(".")[1] == "PV_RW":            
                main_input_dict_1={
                    "tagName": TagName,
                    "dataScienceTagName": "null",
                    "timeStampsToValueMap":{final_timestamp:final_value},
                    "tagConfiguration": {}
                    }
                main_output_dict_1={
                "tagName": main_tag+".CL_ECPR" ,
                "dataScienceTagName": "null",
                "outputConfiguration": "null"
                }
                InputDataList.append(main_input_dict_1)              
                OutputDataList.append(main_output_dict_1)
            elif i[0].split(".")[1] == "STDEV" :                    
                main_input_dict_2={
                    "tagName": TagName,
                    "dataScienceTagName": "null",
                    "timeStampsToValueMap":{final_timestamp:final_value},
                    "tagConfiguration": {}
                    }
                main_output_dict_2={
                "tagName": main_tag+".PV_AG" ,
                "dataScienceTagName": "null",
                "outputConfiguration": "null"
                }
                InputDataList.append(main_input_dict_2)
                OutputDataList.append(main_output_dict_2)   
            

            
#     print(OutputDataList)
    return InputDataList,OutputDataList
#     print(tags_dict4)

In [31]:
def final_json_file(main_config,InputDataList,OutputDataList):
    Final_dict={
        "JobId": "d29ddf43-7df4-490b-96fc-99f41eacbb1c",
        "ModelId": "model-cps-hbwnarea01nap01-pareto1",
        "CalculationType": "GED_DR_MASS_TEST",
        "Configuration":main_config,
        "InputDataList":InputDataList,
        "OutputDataList": OutputDataList
    }
    return Final_dict
    

In [32]:
import openpyxl
import xlrd
import xlsxwriter
import pandas as pd
import time


input_file="PDME SSD v3.0-beta6.xlsm"
rb = xlrd.open_workbook(input_file)
sheetNames = rb.sheet_names()


critical_sensor_tags=[]
edge_name_keys=[]
edge_name_values=[]
edge_asset_keys=[]
edge_asset_values=[]
edge_asset_values_1=[]
tags_dict= {}
a=b=1

pros= subprocess.Popen(["C:\\Windows\\syswow64\\WindowsPowerShell\\v1.0\\powershell.exe",".\\MR_phddata_pull.PS1"],stdout=sys.stdout)
pros.communicate()

for name in sheetNames:
    if name== "Data Extraction":
        sheet1= rb.sheet_by_name(name)
        rows=sheet1.nrows
        for i in range(rows):
            if sheet1.cell_value(i,4) != 'x' and sheet1.cell_value(i,5) != '' and sheet1.cell_value(i,5) != 'Sensor Tags' :
                critical_sensor_tags.append(sheet1.cell_value(i,5))

    if name=="Data Reconciliation":
        
        sheet=rb.sheet_by_name(name)
        for i in range(sheet.nrows):
            
            edge_name=sheet.cell_value(i,1)+sheet.cell_value(i,2)
            if edge_name != '' and edge_name !='Start NodeEnd Node': 
                edge_name_keys.append("EDGE%d_NAME"%(a))
                edge_name_values.append(edge_name)
                a+=1
            else:
                pass
            
            if sheet.cell_value(i,1) != '' and sheet.cell_value(i,4) != 'Tag Name':
                edge_asset_keys.append("EDGE%d_ASSET"%(b))
                edge_asset_values.append(sheet.cell_value(i,4))
                edge_asset_values_1.append(sheet.cell_value(i,4).split(".")[0])
                b+=1
                
            
#         print(edge_name_keys)
        series_1=pd.Series(edge_name_keys)
#         print(edge_name_values)
        series_2=pd.Series(edge_name_values)
#         print(edge_asset_keys)
        series_3=pd.Series(edge_asset_keys)
#         print(edge_asset_values) 
        series_4=pd.Series(edge_asset_values_1)
        Data1={"edge_names":series_1,"edge_name_values":series_2,"edge_asset":series_3,"edge_asset_values":series_4}
#         print(Data1)
        dframe1= pd.DataFrame(Data1)
#         print(dframe1)
        
        dict_1=pd.Series(dframe1.edge_name_values.values,index= dframe1.edge_names).to_dict()
        dict_2=pd.Series(dframe1.edge_asset_values.values,index=dframe1.edge_asset).to_dict()
        res_dict={**dict_1,**dict_2}
        res_dict["CONFIG_FILE"]="TestConfiguration"
        res_dict["REACTION_INDICATOR"]="0"
        res_dict["TERMINAL_INDICATOR"]="1"
        res_dict["INTERVAL"]="30"
        res_dict["TOTAL_COMPONENTS"]="0"
        res_dict["MODEL_SERVICE_TOKEN"]="<Token>"
        res_dict["UNIT_STEADY_STATE_TAG"]="U0300_01.SSD1_ST"
        res_dict["UNIT_STEADY_STATE_REQUIREMENT"]="0.5"
        res_dict["INTERUNIT_INDICATOR"]="0"
        res_dict["MODEL_SERVICE_ENDPOINT"]="<URL>"
        res_dict["SAMPLE_DATA_FREQUENCY"]="100"
        res_dict["COMPONENT_INDICATOR"]="0"
        res_dict["DELAY"]="150"
        
        
#         print(res_dict)
#         tag_dict = dframe1.set_index('edge_names',"edge_asset")["edge_name_values","edge_asset_values"].to_dict()
        
        

                
#                 if sheet.cell_value(i,4)=="" or sheet.cell_value(i,4)==tag_value :
#                     critical_tags_window_size_values.append(sheet.cell_value(i,10))
#                     series_4=pd.Series(critical_tags_window_size_values)
#                     critical_tags_window_size_keys.append("CRITICAL_TAG_%d_WINODW_SIZE"%(b))
#                     series_3=pd.Series(critical_tags_window_size_keys)
#                     b+=1
#                     critical_tags_liklihood_values.append(sheet.cell_value(i,13)*100)
#                     series_5=pd.Series(critical_tags_liklihood_values)
#                     critical_tags_likelihood_keys.append("CRITICAL_TAG_%d_LIKELIHOOD_REQ"%(c))
#                     series_6=pd.Series(critical_tags_likelihood_keys)        
#                     c+=1     
                    
                    
         
#         Data2={"window_size_tag_names":series_3,"window_size_tag_values":series_4}
#         Data3={"likelihood_tag_names":series_6,"likelihood_values":series_5}
#         dframe2=pd.DataFrame(Data2)
#         #convert dataframe to dictionary
#         tags_dict2=dframe2.set_index('window_size_tag_names')["window_size_tag_values"].to_dict()
#         dframe3=pd.DataFrame(Data3)
#         tags_dict3=dframe3.set_index('likelihood_tag_names')["likelihood_values"].to_dict()
        
# #         print(dframe2)
        
# #         tags_dict2=dframe.set_index('window_size_tag_names').to_dict()
# #         print(tags_dict2)
# #         print(tags_dict3)
        
# main_config=json_input(sheet,tags_dict,tags_dict2,tags_dict3)
# # print(main_config) 

# # TagName,tags_dict4  =
InputDataList,OutputDataList=inputDataList(critical_sensor_tags,edge_asset_values)
# print(TagName)

Final_dict=final_json_file(res_dict,InputDataList,OutputDataList)
# print(Final_dict)
import json

json_object=json.dumps(Final_dict,indent=4)

with open("MR_inputJSONfile.json","w") as outfile:
    outfile.write(json_object)


('TagName', 'TimeStamp', 'Value')
('XI_23005_01_MR.PV_RW', '02-26-2021 18:57:39', '790230.1')
Fetching data for  XI_23005_01_MR.PV_RW
('XI_23014_01_MR.PV_RW', '02-26-2021 18:57:39', '267938.8')
Fetching data for  XI_23014_01_MR.PV_RW
('XI_23015_01_MR.PV_RW', '02-26-2021 18:57:39', '548872.3')
Fetching data for  XI_23015_01_MR.PV_RW
('XI_23032_01_MR.PV_RW', '02-26-2021 18:57:39', '1838711')
Fetching data for  XI_23032_01_MR.PV_RW
('XI_23024_01_MR.PV_RW', '02-26-2021 18:57:39', '362227.9')
Fetching data for  XI_23024_01_MR.PV_RW
('XI_23026_01_MR.PV_RW', '02-26-2021 18:57:39', '180051.2')
Fetching data for  XI_23026_01_MR.PV_RW
('XI_23029_01_MR.PV_RW', '02-26-2021 18:57:39', '270076.2')
Fetching data for  XI_23029_01_MR.PV_RW
('XI_23049_01_MR.PV_RW', '02-26-2021 18:57:39', '13427.33')
Fetching data for  XI_23049_01_MR.PV_RW
('XI_23050_01_MR.PV_RW', '02-26-2021 18:57:39', '83989.97')
Fetching data for  XI_23050_01_MR.PV_RW
('XI_23293_01_MR.PV_RW', '02-26-2021 18:57:39', '2050.29')
Fetching